In [1]:
import numpy as np
from astropy.io import fits
from scipy.interpolate import interp1d
import math as m
import scipy.constants as p
import pandas as pd
import matplotlib.pyplot as plt
import timeit
import sys
sys.path.append("..")
import physics
sys.path.append("../..")
from spec.spectrum import *
#import Electrons_v1
#%run 'Electrons_v1.ipynb'
import xsecs
np.set_printoptions(precision=4)

In [2]:
#Test spectrum
eng1 = np.logspace(0.,3.,num = 500)
dNdE1 = np.logspace(0.,5.,num = 500)

eng2   = np.logspace(0.,3.,num=2)
dNdE2 = np.logspace(0.,5.,num=2)

testMat = np.array([[1,2],[2,1]])

rs = 2000.

specObj1 = Spectrum(eng1,dNdE1,rs)
specObj2 = Spectrum(eng2,dNdE2,rs)

ion_frac=0.5

In [3]:
eng1 = np.logspace(0.,3.,num = 500)
dNdE1 = np.logspace(0.,5.,num = 500)
rs=1000
E_spec = Spectrum(eng1,dNdE1,rs)
photon_spec = Spectrum(E_spec.eng,np.zeros(len(E_spec.eng)),rs)
TM=np.zeros([len(photon_spec.eng)+2,len(E_spec.eng)]) #possibly make pandas
Transfer_Matrix=pd.DataFrame(TM) #or np.zeros
#print(Transfer_Matrix)
print(np.array([E_spec.eng[330]]))
print(xsecs.ionize_cs(np.array([E_spec.eng[100]]),np.ones(1)))

[ 96.3758]
[ 0.]


In [6]:
eng1 = np.logspace(0.,3.,num = 500)
dNdE1 = np.logspace(0.,5.,num = 500)
rs=1000
ion_frac=[0.5,0.5,0.5]
E_spec = Spectrum(eng1,dNdE1,rs)


shift = Spectrum((E_spec.eng+xsecs.heating_dE(E_spec.eng, ion_frac[0], rs, nH=physics.nH)), E_spec.dNdE, rs)
shift.rebin(E_spec.eng)
print(shift.toteng())
print(E_spec.toteng())

print(shift.totN())
print(E_spec.totN())



27970736981.1
27970742355.1
38196427.8074
38196417.9184


In [25]:
        print(E_spec.eng[5])
        dE = xsecs.heating_dE(E_spec.eng[5], 0.5, rs, nH=physics.nH)
    
        delta_dNdE = np.zeros(len(E_spec.dNdE))
        np.put(delta_dNdE, 5, E_spec.dNdE[5])
        delta = Spectrum(E_spec.eng, delta_dNdE, rs)
        print(delta.totN())
   
        shift_delta_eng = E_spec.eng+dE
        delta.shift_eng(shift_delta_eng)
        
        delta.rebin(E_spec.eng)
        print(delta.totN())
        
        

1.07166764803
0.0166493283539
0.0
0.0166493283539


In [21]:
def electron_low_E(E_spec, rs, ion_frac=[0.5,0.5,0.5], den=[1,1,1], dt=1, all_outputs=False): #update preset density values
    #V1
    #E_spec=electron spectrum class object, rs= current redshift, 
    #ion_frac= H_frac, He_frac, Hep_frac, density= H_den, He_den, Hep_den
    #all_outputs controls if function returns transfer matrix (False) or transfer matrix, photon spectrum, ionization fraction, heating loss (True)
    
    #Initialize photon spectrum (ask about length)
    photon_spec = Spectrum(E_spec.eng,np.zeros(len(E_spec.eng)),rs)
    
    #initialize transfer matrix
    #columns match E_spec
    #rows are 0:heating, 1:ionization, 2->len(photon_spec)+1:photons
    TM=np.zeros([len(photon_spec.eng)+2,len(E_spec.eng)]) #possibly make pandas
    Transfer_Matrix=pd.DataFrame(TM) #read more pd
    
    #check density calculation...
    #insure ion_frac is correct for den[2]
    den[0]=(physics.nH*(1-ion_frac[0]))/(rs)**3
    den[1]=(physics.nHe*(1-ion_frac[1])*(1-ion_frac[2]))/(rs)**3
    den[2]=(physics.nHe*(1-ion_frac[1])*(ion_frac[2]))/(rs)**3 #unsure about how ionfrac works for helium
    
    
    #HEATING
    heat_vector=np.zeros(len(E_spec.eng))
    
    
    #main loop to fill transfer matrix
        #change zeros if heating becomes cross section
    for n in range(len(E_spec.eng)):
        
        
        #Find xsecs with Electrons_v1
        xsec=np.zeros(9)
        #ionization
        xsec[0] = xsecs.ionize_cs(np.array([E_spec.eng[n]]),np.ones(1)) #H ion
        xsec[1] = xsecs.ionize_cs(np.array([E_spec.eng[n]]),2*np.ones(1)) #He ion
        xsec[2] = xsecs.ionize_cs(np.array([E_spec.eng[n]]),3*np.ones(1)) #Hep ion
        #excitation
        #xsec[3] = excite_cs(np.array([E_spec.eng[n]]),1,1) #H-s excite
        #xsec[4] = excite_cs(np.array([E_spec.eng[n]]),2,1) #He-s excite
        #xsec[5] = excite_cs(np.array([E_spec.eng[n]]),3,1) #Hep-s excite
        #xsec[6] = excite_cs(np.array([E_spec.eng[n]]),1,2) #H-p excite
        #xsec[7] = excite_cs(np.array([E_spec.eng[n]]),2,2) #He-p excite
        #xsec[8] = excite_cs(np.array([E_spec.eng[n]]),3,2) #Hep-p excite
            #xsec[9] = thermalize_cs(numpy.array([E_spec.eng[n]])) #heating
            
        #Find probabilities from xsecs
        #find velocity to calculate rate
        v=np.sqrt(2*np.array([E_spec.eng[n]])/physics.me) #velocity
        
        #multipliers
        mult=np.zeros(3)
        mult[0]= v*den[0] #H
        mult[1]= v*den[1] #He
        mult[2]= v*den[2] #Hep
            #mult[3]= #heating (v*electron_density)?
        prob=np.zeros(9)
        #ionization
        ###possibly remove
        prob[0] = xsec[0]*mult[0] #H ion
        prob[1] = xsec[1]*mult[1] #He ion
        prob[2] = xsec[2]*mult[2] #Hep ion
        #excitation
        prob[3] = xsec[3]*mult[0] #H-s excite
        prob[4] = xsec[4]*mult[1] #He-s excite
        prob[5] = xsec[5]*mult[2] #Hep-s excite
        prob[6] = xsec[6]*mult[0] #H-p excite
        prob[7] = xsec[7]*mult[1] #He-p excite
        prob[8] = xsec[8]*mult[2] #Hep-p excite
            #prob[9] = xsec[9]*mult[3] #heating
        
        
        #Call HEAT related things
        dE = xsecs.heating_dE(E_spec.eng[n], 0.5, rs, nH=physics.nH)
        delta_dNdE = np.zeros(len(E_spec.dNdE))
        np.put(delta_dNdE, n, E_spec.dNdE[n])
        delta = Spectrum(E_spec.eng, delta_dNdE, rs)
        shift_delta_eng = E_spec.eng+dE
        delta.shift_eng(shift_delta_eng)
        delta.rebin(E_spec.eng)
        
        heating_frac = delta.eng[n]/E_spec.eng[n]
        
        
        prob_s=np.zeros((3,n))
        #singly diff ionization prob from xsec
        for i in range(n):
            prob_s[0,i]=mult[0]*ionize_s_cs(E_spec.eng[n],E_spec.eng[i],1) #H
            prob_s[1,i]=mult[1]*ionize_s_cs(E_spec.eng[n],E_spec.eng[i],2) #He
            prob_s[2,i]=mult[2]*ionize_s_cs(E_spec.eng[n],E_spec.eng[i],3) #Hep
        #sum probabilities from singly diff 
        prob_atom_ionize_s = sum(prob_s, axis=1) #vector giving prob sum for each element
        
        print(prob[0]-prob_atom_ionize_s[0])
        print(prob[1]-prob_atom_ionize_s[1])
        print(prob[2]-prob_atom_ionize_s[2])
        
        
        

In [ ]:
mult=array([1,2,3])
prob_s=numpy.zeros((3,5))
for i in range(5):
    prob_s[0,i]=mult[0]*ionize_s_cs(E_spec.eng[5],E_spec.eng[i],1) #H
    prob_s[1,i]=mult[1]*ionize_s_cs(E_spec.eng[5],E_spec.eng[i],2) #He
    prob_s[2,i]=mult[2]*ionize_s_cs(E_spec.eng[5],E_spec.eng[i],3) #Hep
#sum probabilities from singly diff 
prob_atom_ionize_s = sum(prob_s, axis=1)
print(prob_s)
print(prob_atom_ionize_s)

prob_s[0,i]=mult[0]*ionize_s_cs(E_spec.eng[5],E_spec.eng[i],1)


In [22]:
electron_low_E(E_spec, rs, ion_frac=[0.5,0.5,0.5], den=[1,1,1], all_outputs=False)


TypeError: sum() takes no keyword arguments